In [4]:
from bs4 import BeautifulSoup
from scrape_craigslist import Scrape
from pandas import DataFrame
from time import sleep
import urllib.request
import pandas as pd
import numpy as np
import requests
import datetime
import os
import io
import re

In [29]:
location='chico'

In [30]:
url = f'https://{location}.craigslist.org/d/cars-trucks-by-owner/search/cto'
Links=[]
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
total_posts=int(soup.find("span", {"class": "total"}).text)
fraction=total_posts%120

In [31]:

final_links=[]

for i in range(0,total_posts-fraction,120):
    try:
        url = f'https://{location}.craigslist.org/d/cars-trucks-by-owner/search/cto?s={i}'
        Links=[]
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
        for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
            Links.append(link.get('href'))
    
        for i in range(9, 249):
            if i%2==0:
                final_links.append(Links[i])
    except:
        continue
            

In [32]:
len(final_links)

240

In [33]:
df_=Scrape(final_links)


In [34]:
df_final=df_.dropna(axis=0, thresh=2, subset=['year:','make:','model:','trim:'], inplace=False)

In [35]:
ct = datetime.datetime.now() 
name=str(ct)
date=name[0:-7].replace(":",'_')
date.replace(" ",'_')
df_final.to_csv(rf'C:\Users\Silmi\Desktop\UsedCar-PriceEstimator\data\craigslist_{location}_{date}.csv')